In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sys
sys.path.append("../")
import src.Resources as src

In [2]:
movies = pd.read_csv('../Data/Movies_clean.csv', index_col= 0)  
books = pd.read_csv('../Data/Books_clean.csv', index_col= 0)  

In [3]:
movies

,Title,Image,Genre,Description,Rating,Votes,Directors,Stars
0,Dungeons & Dragons: Honor entre ladrones,https://m.media-amazon.com/images/M/MV5BZjAyMG...,Action|Adventure|Comedy,A charming thief and a band of unlikely advent...,7.6,35423,John Francis Daley| Jonathan Goldstein,Chris Pine| Michelle Rodriguez| Regé-Jean Pag...
1,John Wick 4,https://m.media-amazon.com/images/M/MV5BMDExZG...,Action|Crime|Thriller,John Wick uncovers a path to defeating The Hig...,8.3,106008,Chad Stahelski,Keanu Reeves| Laurence Fishburne| George Geor...
2,Criminales a la vista,https://m.media-amazon.com/images/S/sash/4Fyxw...,Action|Comedy|Crime,Full-time detectives Nick and Audrey are strug...,5.6,30998,Jeremy Garelick,Adam Sandler| Jennifer Aniston| Mark Strong| ...
3,Avatar: El sentido del agua,https://m.media-amazon.com/images/S/sash/4Fyxw...,Action|Adventure|Fantasy,Jake Sully lives with his newfound family form...,7.7,349677,James Cameron,Sam Worthington| Zoe Saldana| Sigourney Weave...
4,Criminales en el mar,https://m.media-amazon.com/images/S/sash/4Fyxw...,Action|Comedy|Crime,A New York cop and his wife go on a European v...,6.0,146485,Kyle Newacheck,Adam Sandler| Jennifer Aniston| Luke Evans| T...
...,...,...,...,...,...,...,...,...
130080,Atop the Fourth Wall: The Movie,https://m.media-amazon.com/images/S/sash/4Fyxw...,Comedy|Sci-Fi|Thriller,A witty comic book reviewer gears up to save t...,3.2,303,Jessica Kitrick| Lewis Lovhaug,Nash Bozard| Teresa Dietzinger| Brad Jones| M...
130081,Alien Reign of Man,https://m.media-amazon.com/images/S/sash/4Fyxw...,Fantasy|Sci-Fi,"Stranded on a distant planet, a team must figh...",2.1,545,Tristan Price,Khu| Torrei Hart| Deanna Grace Congo| Stacey ...
130082,Opium,https://m.media-amazon.com/images/S/sash/4Fyxw...,Comedy|Sci-Fi,Five short stories attempts to examine the rel...,8.4,7,Aman Sachdeva,Anirban Bhattacharyya| Manu Rishi Chadha| Sha...
130083,Il prato macchiato di rosso,https://m.media-amazon.com/images/S/sash/4Fyxw...,Comedy|Horror|Mystery,A group of rich decadents pick up wanderers fr...,5.2,202,Riccardo Ghione,Marina Malfatti| Enzo Tarascio| Daniela Carol...


In [4]:
movies["Directors"]

0          John Francis Daley| Jonathan Goldstein 
1                                  Chad Stahelski 
2                                 Jeremy Garelick 
3                                   James Cameron 
4                                  Kyle Newacheck 
                            ...                   
130080             Jessica Kitrick| Lewis Lovhaug 
130081                              Tristan Price 
130082                              Aman Sachdeva 
130083                            Riccardo Ghione 
130084                               Edward Conna 
Name: Directors, Length: 130085, dtype: object

In [5]:
movies.columns = ["Title","Image","Genre","Description","Rating","Votes","Directors","Stars"]
books.columns = ["Author","Image","Description","Rating","Votes","Title","Genre"]

In [6]:
movies["Stars"].fillna("Nostars",inplace=True)
movies.drop_duplicates(inplace=True)
movies = movies.drop_duplicates(subset="Title")
movies = movies.reset_index()
movies = movies.drop("index",axis=1)

books["Description"].fillna("No description",inplace=True)
books.drop_duplicates(inplace=True)
books = books.drop_duplicates(subset="Title")
books = books.reset_index()
books = books.drop("index",axis=1)

In [7]:
director = movies["Directors"].apply(lambda x: x.split("|"))
m_genre = movies["Genre"].apply(lambda x: x.split("|"))
stars = movies["Stars"].apply(lambda x: x.split("|"))
b_genre = books["Genre"].apply(lambda x: x.split("|"))

In [8]:
n_directors = len(director.explode().unique())
movie_genres = len(m_genre.explode().unique())
movie_stars = len(stars.explode().unique())
book_genre = len(b_genre.explode().unique())
n_authors = books["Author"].nunique()
n_titles = books["Title"].nunique()
n_titles_m = movies["Title"].nunique()

In [9]:
Bignumbers = pd.DataFrame([n_authors], columns=["Book_Authors"])
Bignumbers["Movie_directors"] = n_directors
Bignumbers["Movie_stars"] = movie_stars
Bignumbers["Book_titles"] = n_titles
Bignumbers["Movie_titles"] = n_titles_m
Bignumbers["Book_genres"] = book_genre
Bignumbers["Movie_genres"] = movie_genres

In [10]:
Bignumbers

,Book_Authors,Movie_directors,Movie_stars,Book_titles,Movie_titles,Book_genres,Movie_genres
0,5202,41338,129173,12473,77900,22,28


In [11]:
movies_1 = movies[movies["Votes"]>1000]
books_1 = books[books["Votes"]>1000]

In [12]:
max_m =movies_1["Rating"].max()
min_m = movies_1["Rating"].min()
median_m = movies_1["Rating"].median()
mean_m = movies_1["Rating"].mean()

In [13]:
max_b = books_1["Rating"].max()
min_b = books_1["Rating"].min()
median_b = books_1["Rating"].median()
mean_b = books_1["Rating"].mean()

In [14]:
metric = ["Max","Min","Median","Mean","Max","Min","Median","Mean"]
rating = [max_m,min_m,median_m,mean_m,max_b,min_b,median_b,mean_b]
type = ["Movie","Movie","Movie","Movie","Book","Book","Book","Book"]


In [15]:
Data = pd.DataFrame({"Metric": metric, "Rating": rating, "Type": type})

In [16]:
fig = px.bar(Data, x="Metric",y="Rating",color="Type",barmode="group")
fig.show()

In [17]:
books['Genre'] = books['Genre'].apply(src.map_genre)

In [18]:
books.dropna(inplace=True)
movies.dropna(inplace=True)
books["Genre"] = books["Genre"].apply(lambda x: x.split("|"))
movies["Genre"] = movies["Genre"].apply(lambda x : x.split("|"))

In [19]:
genre_books = books["Genre"].explode().value_counts().reset_index()
genre_movies = movies["Genre"].explode().value_counts().reset_index()

In [20]:
genre_books["Type"]= "Book"
genre_movies["Type"] = "Movie"

In [21]:
genre_total = pd.concat([genre_books,genre_movies])

In [22]:
df = genre_total

fig = px.bar(
    genre_total,
    x = "index",
    y = "Genre",
    color = "Type",
    barmode="group",
    color_discrete_map = {"Book": "#44CF6C", "Movie": "#0E7C7B"})

fig.update_layout(
    plot_bgcolor = "#E1D5D7",
    paper_bgcolor = "#F7EBEC",
    font = dict(family = "Arial", color = "#00072D"))
    
fig.show()

In [23]:
movies_2 = movies[movies["Votes"]>80000]
books_2 = books[books["Votes"]>80000]

In [24]:
movies_2.head(1)

,Title,Image,Genre,Description,Rating,Votes,Directors,Stars
1,John Wick 4,https://m.media-amazon.com/images/M/MV5BMDExZG...,"[Action, Crime, Thriller]",John Wick uncovers a path to defeating The Hig...,8.3,106008,Chad Stahelski,Keanu Reeves| Laurence Fishburne| George Geor...


In [25]:
books_2.head(1)

,Author,Image,Description,Rating,Votes,Title,Genre
0,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Romeo and Juliet, Shakespeare creates a vio...",7.48,2462752,Romeo and Juliet,[Romance]


In [26]:
books_exploded = pd.DataFrame({"Image":books_2["Image"],"Title": books_2["Title"],"Author":books_2["Author"], "Rating": books_2["Rating"],"Description":books_2["Description"],"Genre":books_2["Genre"]})
movies_exploded = pd.DataFrame({"Image":movies_2["Image"],"Title": movies_2["Title"],"Directors":movies_2["Directors"], "Rating": movies_2["Rating"],"Description": movies_2["Description"],"Genre":movies_2["Genre"]})


In [27]:
books_exploded = books_exploded.explode("Genre").reset_index(drop=True)
movies_exploded = movies_exploded.explode("Genre").reset_index(drop=True)

In [28]:
def best_books(genre):
    books_exploded = pd.read_csv("../Data/Books_exploded.csv",index_col=0)
    
    books_genre = books_exploded[books_exploded["Genre"] == genre ]
    top_10_gen = books_genre.sort_values(by="Rating",ascending=False).head(10)
    top_10_gen.drop("Genre",axis=1,inplace=True)
    return top_10_gen

In [29]:
best_books("Thriller")

,Image,Title,Author,Rating,Description
3538,https://images-na.ssl-images-amazon.com/images...,Paint It All Red,S.T. Abby,9.16,"Hush, little baby, don't say a word... Will L..."
3530,https://images-na.ssl-images-amazon.com/images...,All the Lies,S.T. Abby,9.08,"To understand the monsters in the world, you h..."
203,https://images-na.ssl-images-amazon.com/images...,The Green Mile,Stephen King,8.94,"At Cold Mountain Penitentiary, along the lonel..."
3517,https://images-na.ssl-images-amazon.com/images...,Scarlet Angel,S.T. Abby,8.84,"To kill a monster, you have to be twice as mon..."
121,https://images-na.ssl-images-amazon.com/images...,The Godfather,Mario Puzo,8.76,The Godfather—the epic tale of crime and betra...
296,https://images-na.ssl-images-amazon.com/images...,Verity,Colleen Hoover,8.72,Lowen Ashleigh is a struggling writer on the b...
3285,https://images-na.ssl-images-amazon.com/images...,Troubled Blood,Robert Galbraith,8.72,Private Detective Cormoran Strike is visiting ...
3525,https://images-na.ssl-images-amazon.com/images...,Firekeeper's Daughter,Angeline Boulley,8.70,"As a biracial, unenrolled tribal member and th..."
371,https://images-na.ssl-images-amazon.com/images...,Different Seasons,Stephen King,8.70,This Book is in Good Condition. Used Copy With...
3226,https://images-na.ssl-images-amazon.com/images...,A Good Girl's Guide to Murder,Holly Jackson,8.70,"The case is closed. Five years ago, schoolgirl..."


In [30]:
def best_movies(genre):
    movies_exploded = pd.read_csv("../Data/Movies_exploded.csv",index_col=0)
    
    movies_genre = movies_exploded[movies_exploded["Genre"] == genre ]
    
    top_10_gen = movies_genre.sort_values(by="Rating",ascending=False).head(10)
    top_10_gen.drop("Genre",axis=1,inplace=True)
    return top_10_gen

In [31]:
best_movies("Thriller")

,Image,Title,Directors,Rating,Description
5316,https://m.media-amazon.com/images/S/sash/4Fyxw...,El silencio de los corderos,Jonathan Demme,8.6,A young F.B.I. cadet must receive the help of ...
5307,https://m.media-amazon.com/images/S/sash/4Fyxw...,Infiltrados,Martin Scorsese,8.5,An undercover cop and a mole in the police att...
6802,https://m.media-amazon.com/images/S/sash/4Fyxw...,La ventana indiscreta,Alfred Hitchcock,8.5,A wheelchair-bound photographer spies on his n...
6720,https://m.media-amazon.com/images/S/sash/4Fyxw...,Psicosis,Alfred Hitchcock,8.5,"A Phoenix secretary embezzles $40,000 from her..."
5833,https://m.media-amazon.com/images/S/sash/4Fyxw...,Parásitos,Bong Joon Ho,8.5,Greed and class discrimination threaten the ne...
260,https://m.media-amazon.com/images/S/sash/4Fyxw...,El caballero oscuro: La leyenda renace,Christopher Nolan,8.4,"Eight years after the Joker's reign of chaos, ..."
6077,https://m.media-amazon.com/images/S/sash/4Fyxw...,La vida de los otros,Florian Henckel von Donnersmarck,8.4,"In 1984 East Berlin, an agent of the secret po..."
5940,https://m.media-amazon.com/images/S/sash/4Fyxw...,Masacre (Ven y mira),Elem Klimov,8.4,"After finding an old rifle, a young boy joins ..."
6722,https://m.media-amazon.com/images/S/sash/4Fyxw...,Memento,Christopher Nolan,8.4,A man with short-term memory loss attempts to ...
5351,https://m.media-amazon.com/images/S/sash/4Fyxw...,Reservoir Dogs,Quentin Tarantino,8.3,When a simple jewelry heist goes horribly wron...


In [32]:
best_books("Thriller")

,Image,Title,Author,Rating,Description
3538,https://images-na.ssl-images-amazon.com/images...,Paint It All Red,S.T. Abby,9.16,"Hush, little baby, don't say a word... Will L..."
3530,https://images-na.ssl-images-amazon.com/images...,All the Lies,S.T. Abby,9.08,"To understand the monsters in the world, you h..."
203,https://images-na.ssl-images-amazon.com/images...,The Green Mile,Stephen King,8.94,"At Cold Mountain Penitentiary, along the lonel..."
3517,https://images-na.ssl-images-amazon.com/images...,Scarlet Angel,S.T. Abby,8.84,"To kill a monster, you have to be twice as mon..."
121,https://images-na.ssl-images-amazon.com/images...,The Godfather,Mario Puzo,8.76,The Godfather—the epic tale of crime and betra...
296,https://images-na.ssl-images-amazon.com/images...,Verity,Colleen Hoover,8.72,Lowen Ashleigh is a struggling writer on the b...
3285,https://images-na.ssl-images-amazon.com/images...,Troubled Blood,Robert Galbraith,8.72,Private Detective Cormoran Strike is visiting ...
3525,https://images-na.ssl-images-amazon.com/images...,Firekeeper's Daughter,Angeline Boulley,8.70,"As a biracial, unenrolled tribal member and th..."
371,https://images-na.ssl-images-amazon.com/images...,Different Seasons,Stephen King,8.70,This Book is in Good Condition. Used Copy With...
3226,https://images-na.ssl-images-amazon.com/images...,A Good Girl's Guide to Murder,Holly Jackson,8.70,"The case is closed. Five years ago, schoolgirl..."


In [33]:
books_exploded.to_csv("../Data/Books_exploded.csv")
movies_exploded.to_csv("../Data/Movies_exploded.csv")